In [2]:
import os
import openai

%set_env os.environ["OPENAI_API_KEY"] = "key"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [1]:
from llama_index.core import StorageContext,load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.retrievers.bm25 import BM25Retriever

import nest_asyncio
nest_asyncio.apply()

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large",device='cpu')

load_index = load_index_from_storage(StorageContext.from_defaults(persist_dir='./localvecstore/'),embed_model=embed_model)

C:\Users\sukpi\anaconda3\envs\monkbot001\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
vector_retriever = load_index.as_retriever(similarity_top_k=5)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=load_index.docstore, similarity_top_k=10
)

In [5]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    retriever_weights=[0.7, 0.3],
    similarity_top_k=10,
    num_queries=1,  # set this to 1 to disable query generation
    mode="relative_score",
    use_async=True,
    verbose=True,
)

In [8]:
nodes_with_scores = retriever.retrieve(
    "การทำบุญ"
)

In [9]:
for node in nodes_with_scores:
    print(f"Score: {node.score:.2f} - {node.text[:100]}...\n-----")

Score: 1.00 - การทำบุญให้คนที่เสียชีวิตหมดเป็นจำนวนเยอะเขาจะได้บุญไหม...
-----
Score: 0.73 - ทําบุญ กับทําทานต่างกันยังไง? ...
-----
Score: 0.61 - การบวชปฏิบัติ...
-----
Score: 0.57 - ทําบุญอะไรไปสวรรค์ชั้นต่างต่าง...
-----
Score: 0.24 - การทำบุญถวายสังฆทานแล้วคิดว่าอยากได้มีของทำบุญมากมาก ต้องมีเจ้าอาวาสมารับ ต้องได้นั่งแถวหน้า ต้องเป็...
-----
Score: 0.14 - การทำบุญด้วยวิธีใด หรือสิ่งใดที่จะส่งผลบุญกุศลได้ดีให้กับมารดาที่เสียชีวิตไปแล้วให้ได้ดีที่สุด...
-----
Score: 0.10 - ถ้าถวายสังฆทานแล้วพระไม่ได้ใช้ เรายังจะได้บุญ 100% ไหมคะ...
-----
Score: 0.09 - ได้ยินเสียงเปรตมาขอส่วนบุญต้องทําอย่างไรเขาถึงจะได้รับบุญ ...
-----
Score: 0.03 - รื่องการคิดบุญ แล้วก็คาดหวังเหมือนกันว่าจะให้ญาติได้บุญนี้นะครับ เป็นการยึดติดอะไร...
-----
Score: 0.01 - ถ้างั้นเวลาเราทําบุญ คําอธิษฐานที่พระอาจารย์แนะนําว่าเป็นคําอธิษฐานเพื่อกุศลของตนที่ฉลาดที่สุด น่าจะ...
-----
